In [70]:
import re
from IPython.display import SVG, display

In [71]:
def extract_shapes_from_svg(root):
    # Namespace handling
    ns = {'svg': 'http://www.w3.org/2000/svg'}

    # Step 1: Extract styles from <style> tag
    style_text = root.find(".//svg:style", ns).text

    # Parse class styles
    style_dict = {}
    for match in re.finditer(r'\.(\w+)\s*{\s*fill:\s*(#[0-9a-fA-F]+);', style_text):
        class_name, fill_color = match.groups()
        style_dict[class_name] = fill_color

    # Step 2: Extract all shape elements
    shape_data = []

    for elem in root.findall(".//svg:*", ns):
        tag = elem.tag.split('}')[-1]  # strip namespace
        if tag in {"rect", "polygon", "path", "circle"}:
            class_name = elem.attrib.get("class")
            style = style_dict.get(class_name, None)
            shape_info = {
                "tag": tag,
                "class": class_name,
                "style": style,
                "attributes": elem.attrib,
                "element": elem
                
            }
            shape_data.append(shape_info)

        elif tag in {"style", "defs"}:
            continue
        else:
            print(f"Skipping unknown shape type: {tag}")
            continue

    return shape_data



def extract_styles_from_svg(root):
    """
    Extract style information from an SVG root element.
    Returns a dictionary mapping class names to fill colors.
    """
    # Namespace (required for SVG parsing)
    ns = {'svg': 'http://www.w3.org/2000/svg'}

    # Find the <style> tag
    style_element = root.find(".//svg:style", ns)
    style_text = style_element.text if style_element is not None else ""

    # Clean and extract styles from the CSS block
    style_dict = {}
    matches = re.findall(r'\.(cls-\d+)\s*{\s*fill:\s*(#[0-9a-fA-F]+);', style_text)

    for class_name, fill_color in matches:
        style_dict[class_name] = fill_color

    return style_dict


In [72]:
import xml.etree.ElementTree as ET
import numpy as np
from svgpathtools import parse_path
from shapely.geometry import Polygon
import numpy as np

def get_path_interior_point(d_attr, samples=200):
    try:
        path = parse_path(d_attr)

        # Sample many points along the path
        sampled_points = [segment.point(t) for segment in path for t in np.linspace(0, 1, 10)]
        coords = [(p.real, p.imag) for p in sampled_points]

        if len(coords) < 3:
            raise ValueError("Not enough points to form a polygon.")

        # Ensure it's closed
        if coords[0] != coords[-1]:
            coords.append(coords[0])

        polygon = Polygon(coords)

        # Fix invalid polygons
        if not polygon.is_valid:
            polygon = polygon.buffer(0)

        if not polygon.is_valid or polygon.area == 0:
            raise ValueError("Polygon still invalid or zero-area after fix.")

        # Safe point inside
        pt = polygon.representative_point()
        return pt.x, pt.y, True

    except Exception as e:
        print(f"Error creating polygon from path: {e}")
        path = parse_path(d_attr)
        xmin, xmax, ymin, ymax = path.bbox()
        return (xmin + xmax) / 2, (ymin + ymax) / 2, False


def get_rect_center(attrib):
    x = float(attrib.get("x", 0))
    y = float(attrib.get("y", 0))
    w = float(attrib["width"])
    h = float(attrib["height"])
    return x + w / 2, y + h / 2

def get_polygon_centroid(points_str):
    nums = list(map(float, points_str.strip().split()))
    if len(nums) % 2 != 0:
        raise ValueError("Odd number of coordinates in polygon points.")
    
    points = list(zip(nums[::2], nums[1::2]))  # Pair x, y
    x_vals, y_vals = zip(*points)
    return sum(x_vals) / len(x_vals), sum(y_vals) / len(y_vals)

def get_circle_center(attrib):
    cx = float(attrib["cx"])
    cy = float(attrib["cy"])
    return cx, cy

def add_number_labels_to_svg(root, shapes, style_dict):
    ns = {'svg': 'http://www.w3.org/2000/svg'}
    ET.register_namespace('', ns['svg'])  # ensure output has correct namespace

    # Create style mapping from values to numbers
    style_mapping = {key: str(i) for i, key in enumerate(style_dict.keys())}

    shape_type_color_mapping = {
        "rect": "black",
        "polygon": "orange",
        "circle": "red",
        "path": "blue"
    }

    for idx, shape in enumerate(shapes):
        tag = shape['tag']
        attrib = shape['attributes']

        if tag == "rect":
            cx, cy = get_rect_center(attrib)
        elif tag == "polygon":
            cx, cy = get_polygon_centroid(attrib["points"])
        elif tag == "circle":
            cx, cy = get_circle_center(attrib)
        elif tag == "path":
            cx, cy, is_valid = get_path_interior_point(attrib["d"])
        else:
            print(f"Skipping unknown shape type: {tag}")
            continue

        # Create a new <text> element
        text_elem = ET.Element("text", {
            "x": str(cx),
            "y": str(cy),
            "font-size": "3",
            "fill": "black",
            "text-anchor": "middle",
            "dominant-baseline": "central"
        })
        text_elem.text = style_mapping[shape['attributes']['class']]

        root.append(text_elem)

    return root

def outline_shapes_for_paint_by_numbers(shapes):
    for shape in shapes:
        elem = shape['element']

        # Remove 'class' attribute
        elem.attrib.pop('class', None)

        # Set stroke and remove fill
        elem.attrib['fill'] = 'white'
        elem.attrib['stroke'] = 'lightgray'
        elem.attrib['stroke-width'] = '0.5'

In [ ]:
svg_path = "data/01.svg"

# Load and parse the SVG file
tree = ET.parse(svg_path)
root = tree.getroot()

# Extract styles and shapes from the SVG
style_dict = extract_styles_from_svg(root)
shapes = extract_shapes_from_svg(root)

# Modify SVG in-place
add_number_labels_to_svg(root, shapes, style_dict)
outline_shapes_for_paint_by_numbers(shapes)  # no assignment!

# Save modified SVG
tree.write("labeled_output.svg", encoding="utf-8", xml_declaration=True)

Error creating polygon from path: Polygon still invalid or zero-area after fix.
Error creating polygon from path: Polygon still invalid or zero-area after fix.
Error creating polygon from path: Polygon still invalid or zero-area after fix.
Error creating polygon from path: Polygon still invalid or zero-area after fix.
Error creating polygon from path: Polygon still invalid or zero-area after fix.
